# recreating HGTDB

G+C content, 
codon usage, 
or amino acid compositio

**GC content:**

- We considered genes as extraneous in terms of the G+C content if their G+C(T) content deviated by >1.5sigma from the mean value of their genome or 
- if deviations of G+C(1) and G+C(3) were of the same sign and at least one was >1.5sigma

**Codon Usage:**
- Mahalonobis distance is used as a measure of the distance between the codon usage of a gene (X) and the mean of an organism

- each gene is a vector of 61-D space; rel freq of 61 codons (stop codons not included)

- Mahalonobis distance defined as 
    - dM(X,Xhat) = (X-Xhat)^T * S^-1 * (X-Xhat)

- The covariance S is a 61 X 61 covriance
matrix
    - Sij = sigma( (Xki - Xhati) * (Xkj - Xhatj)  )

- Xhat is mean value for each codon!

important part is 

"We calculated the Mahalanobis distance from each gene to the mean value of its own organism. These distances did not follow a normal distribution, so we could not apply the criteria regarding deviations >1.5 from the mean value to identify extraneous genes from codon usage. Instead we used a Montecarlo procedure"

- generating a random sample of 10,000 sequences from the means and standard deviations of the codon usage of each genome

- The Mahalanobis distances of these sets of random sequences had a normal distribution, and so, we could calculate a mean value and a standard deviation

**Amino acid:**

What large deviations from the mean values of amino acid composition represent is very ambiguous. They may be caused either by functional constraints or by the result of the extraneous codon usage or G+C content of a horizontally transferred gene. We therefore chose the restricting criterion:

We excluded from our set of genes predicted as being acquired by HGT those isolated genes whose derived protein has deviations of >3signa in at least one amino acid content. Only genes included in some of the alien genomic strips could present such deviation.

In [1]:
from data.utils.NCBI.data_loader import NCBIDataLoader
# from data.utils.HGTREE.data_loader import HGTREEDataLoader
# test out combination

aquifex_aeolicus_VF5 = NCBIDataLoader('AE000657')

# GC content deviation

## Part 1
We considered genes as extraneous in terms of the G+C content if their G+C(T) content deviated by >1.5 from the mean value of their genome or if deviations of G+C(1) and G+C(3) were of the same sign and at least one was >1.5

In [33]:
aquifex_aeolicus_VF5.print_genome_summary()

Mean GC Content-> T:43.34303486743043, 1:50.13798736996561, 2:32.20957374710352, 3:47.681543485221965
Std GC content-> T:3.781751997458822, 1:5.145355530842554, 2:4.714476507774925, 3:6.409771046128724
Relative nucleotide frequency: {'G': 0.2430156486938153, 'A': 0.326311309497143, 'C': 0.1927896813922138, 'T': 0.23788336041682787}
Nucleotide Identity: {'A': [0.32453764605741703, 0.34840166655127336, 0.30599461588273863], 'T': [0.17104143163759433, 0.327946500934686, 0.2146621486782033], 'G': [0.34403776212953646, 0.15032764919340058, 0.2346815347585089], 'C': [0.16038316017545218, 0.17332418332064006, 0.24466170068054915]}
Dinucleotide Identity: {'AA': [0.1301107368746869, 0.1168316791359347, 0.12895638733956052], 'AG': [0.06074237283993859, 0.09573709868573779, 0.10245319942248912], 'AT': [0.09167253815107296, 0.03736911341264249, 0.027861438116125303], 'AC': [0.04201199819171856, 0.09846377531695834, 0.04672217776175844], 'TA': [0.04333359127137661, 0.09658218516964856, 0.0530980119

In [34]:
# gene - genome
print(" Check if gene.GCT - genome.GCT deviates more than 1.5sigma")
print(f" GCT: {abs(aquifex_aeolicus_VF5['aq_171']['GCT'] - aquifex_aeolicus_VF5.mean_GCT)}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GCT}")
print("\n")
print(" Check if either gene.GC1 - genome.GC1 and gene.GC3 - genome.GC3 has same sign")
print( " AND at least one of them is more than 1.5sigma")
print(f" GC1: {abs(aquifex_aeolicus_VF5['aq_171']['GC1'] - aquifex_aeolicus_VF5.mean_GC1)}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GC1}")
print(f" GC3: {abs(aquifex_aeolicus_VF5['aq_171']['GC3'] - aquifex_aeolicus_VF5.mean_GC3)}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GC3}")



 Check if gene.GCT - genome.GCT deviates more than 1.5sigma
 GCT: 7.310646203462824  > 1.5sigma: 5.672627996188233


 Check if either gene.GC1 - genome.GC1 and gene.GC3 - genome.GC3 has same sign
 AND at least one of them is more than 1.5sigma
 GC1: 6.818149313285446  > 1.5sigma: 7.718033296263831
 GC3: 7.195713525707795  > 1.5sigma: 9.614656569193086


let us loop this damn thing

27.09.2023 -> added absolute sign... important

https://en.wikipedia.org/wiki/Standard_deviation

In [35]:
# need to rethink gene listing i guess

list_of_extraneous_genes = []
for i in aquifex_aeolicus_VF5.genes:
    dev_GCT = aquifex_aeolicus_VF5[i]['GCT'] - aquifex_aeolicus_VF5.mean_GCT
    
    dev_GC1 = aquifex_aeolicus_VF5[i]['GC1'] - aquifex_aeolicus_VF5.mean_GC1
    dev_GC3 = aquifex_aeolicus_VF5[i]['GC3'] - aquifex_aeolicus_VF5.mean_GC3
    equal_sign_check = dev_GC1*dev_GC3
    
    if len(aquifex_aeolicus_VF5[i]['sequence']) > 300:
        if abs(dev_GCT) > (1.5*aquifex_aeolicus_VF5.std_GCT):
            #print(f'HGT found at {i}')
            list_of_extraneous_genes.append(i)
        elif (equal_sign_check > 0):
            if abs(dev_GC1) > (1.5*aquifex_aeolicus_VF5.std_GC1):
                #print(f'HGT found at {i}')
                list_of_extraneous_genes.append(i)
            elif abs(dev_GC3) > (1.5*aquifex_aeolicus_VF5.std_GC3):
                #print(f'HGT found at {i}')
                list_of_extraneous_genes.append(i)
            else:
                pass
        else:
            pass
        
                

In [36]:
len(list_of_extraneous_genes)

243

In [37]:
list_of_extraneous_genes

['aq_005',
 'aq_013',
 'aq_018',
 'aq_032',
 'aq_035',
 'aq_050',
 'aq_059',
 'aq_062',
 'aq_081',
 'aq_082',
 'aq_118',
 'aq_125',
 'aq_132',
 'aq_141',
 'aq_156',
 'aq_171',
 'aq_172',
 'aq_182',
 'aq_183',
 'aq_200',
 'aq_220',
 'aq_221',
 'aq_254',
 'aq_255',
 'aq_264',
 'aq_293',
 'aq_326',
 'aq_340',
 'aq_351',
 'aq_359',
 'aq_369',
 'aq_372',
 'aq_376',
 'aq_377',
 'aq_378',
 'aq_380',
 'aq_381',
 'aq_382',
 'aq_383',
 'aq_384',
 'aq_385',
 'aq_386',
 'aq_387',
 'aq_388',
 'aq_402',
 'aq_407',
 'aq_418',
 'aq_434',
 'aq_451',
 'aq_465',
 'aq_473',
 'aq_476',
 'aq_488',
 'aq_503',
 'aq_504',
 'aq_505',
 'aq_506',
 'aq_507',
 'aq_509',
 'aq_510',
 'aq_511',
 'aq_515',
 'aq_516',
 'aq_518',
 'aq_519',
 'aq_520',
 'aq_522',
 'aq_531',
 'aq_558',
 'aq_573',
 'aq_585',
 'aq_599',
 'aq_626',
 'aq_629',
 'aq_645',
 'aq_647',
 'aq_652',
 'aq_662',
 'aq_666',
 'aq_673',
 'aq_678',
 'aq_706',
 'aq_711',
 'aq_734',
 'aq_737',
 'aq_740',
 'aq_771',
 'aq_780',
 'aq_782',
 'aq_812',
 'aq_820',

In [38]:
print(" Check if gene.GCT - genome.GCT deviates more than 1.5sigma")
print(f" GCT: {aquifex_aeolicus_VF5['aq_005']['GCT'] - aquifex_aeolicus_VF5.mean_GCT}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GCT}")
print("\n")
print(" Check if either gene.GC1 - genome.GC1 and gene.GC3 - genome.GC3 has same sign")
print( " AND at least one of them is more than 1.5sigma")
print(f" GC1: {aquifex_aeolicus_VF5['aq_005']['GC1'] - aquifex_aeolicus_VF5.mean_GC1}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GC1}")
print(f" GC3: {aquifex_aeolicus_VF5['aq_005']['GC3'] - aquifex_aeolicus_VF5.mean_GC3}  > 1.5sigma: {1.5*aquifex_aeolicus_VF5.std_GC3}")

 Check if gene.GCT - genome.GCT deviates more than 1.5sigma
 GCT: 6.246456101370882  > 1.5sigma: 5.672627996188233


 Check if either gene.GC1 - genome.GC1 and gene.GC3 - genome.GC3 has same sign
 AND at least one of them is more than 1.5sigma
 GC1: 11.930978147275773  > 1.5sigma: 7.718033296263831
 GC3: 2.3184565147780347  > 1.5sigma: 9.614656569193086


## Part 2
We also ran an 11-gene window through each genome. Five or more extraneous genes in a given window indicated the presence of an alien genomic strip. Finally, we filtered these strips to disregard short isolated segments and to include genes that we did not consider extraneous but that had a deviation of their G+C content of the same sign as the deviation of the strip to which they belong

In [16]:
from Bio.SeqUtils import GC123

In [40]:
list_of_genes = list(aquifex_aeolicus_VF5.genes.keys())

genomic_strips = []


for k in range(len(list_of_genes)-10):
    window = {}
    j = 0
    while j < 11:
        # get window
        locust_tag = list_of_genes[k + j]
        
        # take genes that are more than 300bp
        if len(aquifex_aeolicus_VF5[locust_tag]['sequence'])>300:
            data = aquifex_aeolicus_VF5[locust_tag]
            window[locust_tag] = data
        # iterate        
        j+=1
        
    # count total extraneous genes in window
    extraneous_counter = 0
    for l in window.keys():
        if l in list_of_extraneous_genes:
            extraneous_counter+=1
    
    # check windows with more than or
    # equal to 5 extraneous genes
    if extraneous_counter >=5:
        # add their sequences together
        sequences = ''
        for m in window.keys():
            sequences += window[m]['sequence']
        
        # get standard deviation of strip
        GCT, GC1, GC2, GC3 = GC123(sequences)
        SDT = (GCT - aquifex_aeolicus_VF5.mean_GCT)/aquifex_aeolicus_VF5.std_GCT
        SD1 = (GC1- aquifex_aeolicus_VF5.mean_GC1)/aquifex_aeolicus_VF5.std_GC1
        SD2 = (GC2 - aquifex_aeolicus_VF5.mean_GC2)/aquifex_aeolicus_VF5.std_GC2
        SD3 = (GC3 - aquifex_aeolicus_VF5.mean_GC3)/aquifex_aeolicus_VF5.std_GC3
        
        # tag genes as extraneous if they have equal deviation to the its strip
        for n in window.keys():
            # check only genes not in current list
            if n not in list_of_extraneous_genes:
                check_SDT = window[n]['SDT']*SDT
                check_SD1 = window[n]['SD1']*SD1
                check_SD2 = window[n]['SD2']*SD2
                check_SD3 = window[n]['SD3']*SD3
                
                if (check_SDT > 0) and (check_SD1 > 0) and (check_SD2 > 0) and (check_SD3 > 0):
                    list_of_extraneous_genes.append(n)
            
        
        genomic_strips.append(window)


In [41]:
# the 10s are windows that had bp<300
for i in genomic_strips:
    print(len(i))

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


In [42]:
len(genomic_strips)

101

In [43]:
genomic_strips[0].keys()

dict_keys(['aq_365', 'aq_367', 'aq_369', 'aq_370', 'aq_371', 'aq_372', 'aq_373', 'aq_374', 'aq_376', 'aq_377', 'aq_378'])

In [44]:
genomic_strips[1].keys()

dict_keys(['aq_367', 'aq_369', 'aq_370', 'aq_371', 'aq_372', 'aq_373', 'aq_374', 'aq_376', 'aq_377', 'aq_378', 'aq_380'])

In [46]:
len(list_of_extraneous_genes)

261

In [47]:
len(list_of_genes)

1553

In [48]:
for i in list_of_extraneous_genes:
    print(i)

aq_005
aq_013
aq_018
aq_032
aq_035
aq_050
aq_059
aq_062
aq_081
aq_082
aq_118
aq_125
aq_132
aq_141
aq_156
aq_171
aq_172
aq_182
aq_183
aq_200
aq_220
aq_221
aq_254
aq_255
aq_264
aq_293
aq_326
aq_340
aq_351
aq_359
aq_369
aq_372
aq_376
aq_377
aq_378
aq_380
aq_381
aq_382
aq_383
aq_384
aq_385
aq_386
aq_387
aq_388
aq_402
aq_407
aq_418
aq_434
aq_451
aq_465
aq_473
aq_476
aq_488
aq_503
aq_504
aq_505
aq_506
aq_507
aq_509
aq_510
aq_511
aq_515
aq_516
aq_518
aq_519
aq_520
aq_522
aq_531
aq_558
aq_573
aq_585
aq_599
aq_626
aq_629
aq_645
aq_647
aq_652
aq_662
aq_666
aq_673
aq_678
aq_706
aq_711
aq_734
aq_737
aq_740
aq_771
aq_780
aq_782
aq_812
aq_820
aq_821
aq_833
aq_837
aq_850
aq_862
aq_895
aq_908
aq_917
aq_920
aq_926
aq_940
aq_944
aq_946
aq_950
aq_993
aq_996
aq_1008
aq_1012
aq_1017
aq_1018
aq_1019
aq_1029
aq_1030
aq_1031
aq_1033
aq_1035
aq_1037
aq_1038
aq_1049
aq_1051
aq_1052
aq_1053
aq_1057
aq_1070
aq_1073
aq_1078
aq_1079
aq_1080
aq_1095
aq_1096
aq_1097
aq_1105
aq_1108
aq_1133
aq_1177
aq_1186
aq_1188
aq_

## Putting part 1 and 2 together


In [15]:
def GC_Content_Deviation(genome, return_genomic_strips = False):
    # Init list of extraneous genes
    list_of_extraneous_genes = []
    
    # part 1: GCT, GC1, GC3 deviation
    for gene in genome.genes:
        dev_GCT = genome[gene]['GCT'] - genome.mean_GCT
        
        dev_GC1 = genome[gene]['GC1'] - genome.mean_GC1
        dev_GC3 = genome[gene]['GC3'] - genome.mean_GC3
        equal_sign_check = dev_GC1*dev_GC3
        
        # consider only more than 300bp
        # either gct > 1.5
        # or sign gc1 and gc3 equal and one of is > 1.5
        if len(genome[gene]['sequence']) > 300:
            if abs(dev_GCT) > (1.5*genome.std_GCT):
                #print(f'HGT found at {i}')
                list_of_extraneous_genes.append(gene)
            elif (equal_sign_check > 0):
                if abs(dev_GC1) > (1.5*genome.std_GC1):
                    #print(f'HGT found at {i}')
                    list_of_extraneous_genes.append(gene)
                elif abs(dev_GC3) > (1.5*genome.std_GC3):
                    #print(f'HGT found at {i}')
                    list_of_extraneous_genes.append(gene)
                else:
                    pass
            else:
                pass
    
    # part two: 11 genes window
    list_of_genes = list(genome.genes.keys())
    genomic_strips = []
    
    for k in range(len(list_of_genes)-10):
        window = {}
        j = 0
        while j < 11:
            # get window
            locust_tag = list_of_genes[k + j]
            
            # take genes that are more than 300bp
            if len(genome[locust_tag]['sequence'])>300:
                data = genome[locust_tag]
                window[locust_tag] = data
            # iterate        
            j+=1
            
        # count total extraneous genes in window
        extraneous_counter = 0
        for l in window.keys():
            if l in list_of_extraneous_genes:
                extraneous_counter+=1
        
        # check windows with more than or
        # equal to 5 extraneous genes
        if extraneous_counter >=5:
            # add their sequences together
            window_sequences = ''
            for m in window.keys():
                window_sequences += window[m]['sequence']
            
            # get standard deviation of strip
            GCT, GC1, GC2, GC3 = GC123(window_sequences)
            SDT = (GCT - genome.mean_GCT)/genome.std_GCT
            SD1 = (GC1- genome.mean_GC1)/genome.std_GC1
            SD2 = (GC2 - genome.mean_GC2)/genome.std_GC2
            SD3 = (GC3 - genome.mean_GC3)/genome.std_GC3
            
            # tag genes as extraneous if they have equal deviation to the its strip
            for n in window.keys():
                # check only genes not in current list
                if n not in list_of_extraneous_genes:
                    check_SDT = window[n]['SDT']*SDT
                    check_SD1 = window[n]['SD1']*SD1
                    check_SD2 = window[n]['SD2']*SD2
                    check_SD3 = window[n]['SD3']*SD3
                    
                    if (check_SDT > 0) and (check_SD1 > 0) and (check_SD2 > 0) and (check_SD3 > 0):
                        list_of_extraneous_genes.append(n)
                
            
            genomic_strips.append(window)
    
    if return_genomic_strips:
        return list_of_extraneous_genes, genomic_strips
    else:
        return list_of_extraneous_genes

In [17]:
hgt_candidates=GC_Content_Deviation(aquifex_aeolicus_VF5)
print(len(hgt_candidates))
print(hgt_candidates)

261
['aq_005', 'aq_013', 'aq_018', 'aq_032', 'aq_035', 'aq_050', 'aq_059', 'aq_062', 'aq_081', 'aq_082', 'aq_118', 'aq_125', 'aq_132', 'aq_141', 'aq_156', 'aq_171', 'aq_172', 'aq_182', 'aq_183', 'aq_200', 'aq_220', 'aq_221', 'aq_254', 'aq_255', 'aq_264', 'aq_293', 'aq_326', 'aq_340', 'aq_351', 'aq_359', 'aq_369', 'aq_372', 'aq_376', 'aq_377', 'aq_378', 'aq_380', 'aq_381', 'aq_382', 'aq_383', 'aq_384', 'aq_385', 'aq_386', 'aq_387', 'aq_388', 'aq_402', 'aq_407', 'aq_418', 'aq_434', 'aq_451', 'aq_465', 'aq_473', 'aq_476', 'aq_488', 'aq_503', 'aq_504', 'aq_505', 'aq_506', 'aq_507', 'aq_509', 'aq_510', 'aq_511', 'aq_515', 'aq_516', 'aq_518', 'aq_519', 'aq_520', 'aq_522', 'aq_531', 'aq_558', 'aq_573', 'aq_585', 'aq_599', 'aq_626', 'aq_629', 'aq_645', 'aq_647', 'aq_652', 'aq_662', 'aq_666', 'aq_673', 'aq_678', 'aq_706', 'aq_711', 'aq_734', 'aq_737', 'aq_740', 'aq_771', 'aq_780', 'aq_782', 'aq_812', 'aq_820', 'aq_821', 'aq_833', 'aq_837', 'aq_850', 'aq_862', 'aq_895', 'aq_908', 'aq_917', 'aq_9

# Codon Usage

each gene corresponds to a vector or to a point in the 61-D space whose coordinates are the relative frequency of use of the 61 codons

I am confused as to the definition of relative synonymous codon usage vs relative frequency but here goes:

RSCU: 
- http://genomes.urv.es/CAIcal/tutorial.pdf
- this is also defined as RSCU = S × Nc/Na, where S represents the number of synonymous codons encoding the same amino acid, Nc is the frequency of the codon in the genome, and Na is the relative frequency of the codon for that amino acid (https://www.biostars.org/p/437939/)


relative frequency:
- https://www.biotite-python.org/examples/gallery/sequence/codon_usage.html
- "relative frequencies are calculated by dividing the total number of occurrences of each codon by the total number of occurrences of the respective amino acid"
- is just Nc/Na


Some setups

In [6]:
import numpy as np

they excluded stop codons

In [7]:
CODE_COVARMAT = {
    'ttt': 'F', 'tct': 'S', 'tat': 'Y', 'tgt': 'C',
    'ttc': 'F', 'tcc': 'S', 'tac': 'Y', 'tgc': 'C',
    'tta': 'L', 'tca': 'S', 'ttg': 'L', 'tcg': 'S',
    'tgg': 'W', 'ctt': 'L', 'cct': 'P', 'cat': 'H',
    'cgt': 'R', 'ctc': 'L', 'ccc': 'P', 'cac': 'H',
    'cgc': 'R', 'cta': 'L', 'cca': 'P', 'caa': 'Q',
    'cga': 'R', 'ctg': 'L', 'ccg': 'P', 'cag': 'Q',
    'cgg': 'R', 'att': 'I', 'act': 'T', 'aat': 'N',
    'agt': 'S', 'atc': 'I', 'acc': 'T', 'aac': 'N',
    'agc': 'S', 'ata': 'I', 'aca': 'T', 'aaa': 'K',
    'aga': 'R', 'atg': 'M', 'acg': 'T', 'aag': 'K',
    'agg': 'R', 'gtt': 'V', 'gct': 'A', 'gat': 'D',
    'ggt': 'G', 'gtc': 'V', 'gcc': 'A', 'gac': 'D',
    'ggc': 'G', 'gta': 'V', 'gca': 'A', 'gaa': 'E',
    'gga': 'G', 'gtg': 'V', 'gcg': 'A', 'gag': 'E',
    'ggg': 'G'
}

In [105]:
len(CODE_COVARMAT)

61

## Calculating RSCU and Relative Frequnce Codon

In [25]:
RSCU_aq_171 = {}
RFC_aq_171 = {}
for cds in CODE_COVARMAT:
    RSCU_aq_171[cds.upper()]=0
    RFC_aq_171[cds.upper()]=0

single

In [26]:
aquifex_aeolicus_VF5['aq_171']['cub']['TTT']

9

In [27]:
aquifex_aeolicus_VF5['aq_171']['cub']['TTC']

5

RSCU and RFC

In [31]:
amino_acid = CODE_COVARMAT['ttt']
# get total amount of synonymous codon (S)
S = list(CODE_COVARMAT.values()).count(amino_acid)

# get total count of synonymous codn
Na = 0
for codon in CODE_COVARMAT:
    if CODE_COVARMAT[codon]==amino_acid:
        Na+= aquifex_aeolicus_VF5['aq_171']['cub'][codon.upper()]

Nc = aquifex_aeolicus_VF5['aq_171']['cub']['ttt'.upper()]
# numerator = aquifex_aeolicus_VF5['aq_171']['cub']['ttt'.upper()]
# denominator = (total_count / number_of_synonymous_codon)
if Na == 0:
    RFC = 0
else:
    RFC = (Nc / Na)
    
RSCU = S * RFC
if Na == 0:
    print(0)
else:
    print(RFC)
    print(RSCU)
    print(f"Synonymous S  {S}")
    print(f"freq of codon TTT Nc {Nc}")
    print(f"total freq of F {Na}")

0.6428571428571429
1.2857142857142858
Synonymous S  2
freq of codon TTT Nc 9
total freq of F 14


loop

In [32]:
for cds in CODE_COVARMAT:
    # check amino acid of current codon
    amino_acid = CODE_COVARMAT[cds]
    # get total amount of synonymous codon
    S = list(CODE_COVARMAT.values()).count(amino_acid)
    
    # get total count of synonymous codn
    Na = 0
    for codon in CODE_COVARMAT:
        if CODE_COVARMAT[codon]==amino_acid:
            Na+= aquifex_aeolicus_VF5['aq_171']['cub'][codon.upper()]
    
    Nc = aquifex_aeolicus_VF5['aq_171']['cub'][cds.upper()]
    
    if Nc == 0:
        RFC = float(0)
    else:
        RFC = (Nc / Na)
        
    RSCU = S * RFC

    RFC_aq_171[cds.upper()] = RFC
    RSCU_aq_171[cds.upper()] = RSCU

In [33]:
RSCU_aq_171

{'TTT': 1.2857142857142858,
 'TCT': 1.125,
 'TAT': 1.0,
 'TGT': 0.0,
 'TTC': 0.7142857142857143,
 'TCC': 1.5,
 'TAC': 1.0,
 'TGC': 2.0,
 'TTA': 1.3548387096774193,
 'TCA': 0.75,
 'TTG': 0.7741935483870968,
 'TCG': 1.125,
 'TGG': 0.0,
 'CTT': 1.741935483870968,
 'CCT': 1.3333333333333333,
 'CAT': 1.0,
 'CGT': 0.0,
 'CTC': 1.161290322580645,
 'CCC': 0.6666666666666666,
 'CAC': 1.0,
 'CGC': 0.0,
 'CTA': 0.1935483870967742,
 'CCA': 0.0,
 'CAA': 0.0,
 'CGA': 0.6000000000000001,
 'CTG': 0.7741935483870968,
 'CCG': 2.0,
 'CAG': 2.0,
 'CGG': 0.0,
 'ATT': 0.75,
 'ACT': 0.6666666666666666,
 'AAT': 0.5454545454545454,
 'AGT': 1.5,
 'ATC': 0.15000000000000002,
 'ACC': 0.0,
 'AAC': 1.4545454545454546,
 'AGC': 0.0,
 'ATA': 2.0999999999999996,
 'ACA': 1.3333333333333333,
 'AAA': 0.9032258064516129,
 'AGA': 2.4000000000000004,
 'ATG': 1.0,
 'ACG': 2.0,
 'AAG': 1.096774193548387,
 'AGG': 3.0,
 'GTT': 0.5714285714285714,
 'GCT': 2.0,
 'GAT': 1.1428571428571428,
 'GGT': 1.2307692307692308,
 'GTC': 0.0,
 

In [34]:
RFC_aq_171

{'TTT': 0.6428571428571429,
 'TCT': 0.1875,
 'TAT': 0.5,
 'TGT': 0.0,
 'TTC': 0.35714285714285715,
 'TCC': 0.25,
 'TAC': 0.5,
 'TGC': 1.0,
 'TTA': 0.22580645161290322,
 'TCA': 0.125,
 'TTG': 0.12903225806451613,
 'TCG': 0.1875,
 'TGG': 0.0,
 'CTT': 0.2903225806451613,
 'CCT': 0.3333333333333333,
 'CAT': 0.5,
 'CGT': 0.0,
 'CTC': 0.1935483870967742,
 'CCC': 0.16666666666666666,
 'CAC': 0.5,
 'CGC': 0.0,
 'CTA': 0.03225806451612903,
 'CCA': 0.0,
 'CAA': 0.0,
 'CGA': 0.1,
 'CTG': 0.12903225806451613,
 'CCG': 0.5,
 'CAG': 1.0,
 'CGG': 0.0,
 'ATT': 0.25,
 'ACT': 0.16666666666666666,
 'AAT': 0.2727272727272727,
 'AGT': 0.25,
 'ATC': 0.05,
 'ACC': 0.0,
 'AAC': 0.7272727272727273,
 'AGC': 0.0,
 'ATA': 0.7,
 'ACA': 0.3333333333333333,
 'AAA': 0.45161290322580644,
 'AGA': 0.4,
 'ATG': 1.0,
 'ACG': 0.5,
 'AAG': 0.5483870967741935,
 'AGG': 0.5,
 'GTT': 0.14285714285714285,
 'GCT': 0.5,
 'GAT': 0.5714285714285714,
 'GGT': 0.3076923076923077,
 'GTC': 0.0,
 'GCC': 0.0,
 'GAC': 0.42857142857142855,
 '

In [2]:
aquifex_aeolicus_VF5['aq_171']['RSCU']

{'TTT': 1.2857142857142858,
 'TCT': 1.125,
 'TAT': 1.0,
 'TGT': 0.0,
 'TTC': 0.7142857142857143,
 'TCC': 1.5,
 'TAC': 1.0,
 'TGC': 2.0,
 'TTA': 1.3548387096774193,
 'TCA': 0.75,
 'TAA': 0.0,
 'TGA': 0.0,
 'TTG': 0.7741935483870968,
 'TCG': 1.125,
 'TAG': 3.0,
 'TGG': 0,
 'CTT': 1.741935483870968,
 'CCT': 1.3333333333333333,
 'CAT': 1.0,
 'CGT': 0.0,
 'CTC': 1.161290322580645,
 'CCC': 0.6666666666666666,
 'CAC': 1.0,
 'CGC': 0.0,
 'CTA': 0.1935483870967742,
 'CCA': 0.0,
 'CAA': 0.0,
 'CGA': 0.6000000000000001,
 'CTG': 0.7741935483870968,
 'CCG': 2.0,
 'CAG': 2.0,
 'CGG': 0.0,
 'ATT': 0.75,
 'ACT': 0.6666666666666666,
 'AAT': 0.5454545454545454,
 'AGT': 1.5,
 'ATC': 0.15000000000000002,
 'ACC': 0.0,
 'AAC': 1.4545454545454546,
 'AGC': 0.0,
 'ATA': 2.0999999999999996,
 'ACA': 1.3333333333333333,
 'AAA': 0.9032258064516129,
 'AGA': 2.4000000000000004,
 'ATG': 1.0,
 'ACG': 2.0,
 'AAG': 1.096774193548387,
 'AGG': 3.0,
 'GTT': 0.5714285714285714,
 'GCT': 2.0,
 'GAT': 1.1428571428571428,
 'GGT

In [3]:
aquifex_aeolicus_VF5['aq_171']['RFC']

{'TTT': 0.6428571428571429,
 'TCT': 0.1875,
 'TAT': 0.5,
 'TGT': 0.0,
 'TTC': 0.35714285714285715,
 'TCC': 0.25,
 'TAC': 0.5,
 'TGC': 1.0,
 'TTA': 0.22580645161290322,
 'TCA': 0.125,
 'TAA': 0.0,
 'TGA': 0.0,
 'TTG': 0.12903225806451613,
 'TCG': 0.1875,
 'TAG': 1.0,
 'TGG': 0,
 'CTT': 0.2903225806451613,
 'CCT': 0.3333333333333333,
 'CAT': 0.5,
 'CGT': 0.0,
 'CTC': 0.1935483870967742,
 'CCC': 0.16666666666666666,
 'CAC': 0.5,
 'CGC': 0.0,
 'CTA': 0.03225806451612903,
 'CCA': 0.0,
 'CAA': 0.0,
 'CGA': 0.1,
 'CTG': 0.12903225806451613,
 'CCG': 0.5,
 'CAG': 1.0,
 'CGG': 0.0,
 'ATT': 0.25,
 'ACT': 0.16666666666666666,
 'AAT': 0.2727272727272727,
 'AGT': 0.25,
 'ATC': 0.05,
 'ACC': 0.0,
 'AAC': 0.7272727272727273,
 'AGC': 0.0,
 'ATA': 0.7,
 'ACA': 0.3333333333333333,
 'AAA': 0.45161290322580644,
 'AGA': 0.4,
 'ATG': 1.0,
 'ACG': 0.5,
 'AAG': 0.5483870967741935,
 'AGG': 0.5,
 'GTT': 0.14285714285714285,
 'GCT': 0.5,
 'GAT': 0.5714285714285714,
 'GGT': 0.3076923076923077,
 'GTC': 0.0,
 'GCC':

In [4]:
aquifex_aeolicus_VF5.mean_RSCU

{'TTT': 1.1133693883498172,
 'TCT': 1.089286338139432,
 'TAT': 0.36753716693208593,
 'TGT': 0.7075027345670496,
 'TTC': 0.8763279715986683,
 'TCC': 1.5838193798247604,
 'TAC': 1.619584533003523,
 'TGC': 0.7451695126963116,
 'TTA': 0.9932780937580905,
 'TCA': 0.8800389888653917,
 'TAA': 1.5231809401159047,
 'TGA': 1.1020605280103026,
 'TTG': 0.4745210747949317,
 'TCG': 0.4246119119576557,
 'TAG': 0.37475853187379266,
 'TGG': 0.836445589182228,
 'CTT': 1.5226340538124992,
 'CCT': 1.105517866222857,
 'CAT': 0.3513670116786209,
 'CGT': 0.19394068971627182,
 'CTC': 1.741633160403617,
 'CCC': 1.629929432115504,
 'CAC': 1.4992447075744366,
 'CGC': 0.15655199047787768,
 'CTA': 0.4382802793219187,
 'CCA': 0.5676400799220027,
 'CAA': 0.7041535867957956,
 'CGA': 0.07476811416374893,
 'CTG': 0.8219263578703128,
 'CCG': 0.6788830016494903,
 'CAG': 1.2275914228629277,
 'CGG': 0.09621493217088631,
 'ATT': 0.7352107043831333,
 'ACT': 0.9632271032386273,
 'AAT': 0.5833988913683169,
 'AGT': 0.9807434122

In [5]:
aquifex_aeolicus_VF5.std_RSCU

{'TTT': 0.3840038476200185,
 'TCT': 0.7787113744273603,
 'TAT': 0.33165053093501046,
 'TGT': 0.7325353123462082,
 'TTC': 0.38081071753942397,
 'TCC': 0.9157092897691168,
 'TAC': 0.35512859128007734,
 'TGC': 0.7464744180884368,
 'TTA': 0.5724569162371057,
 'TCA': 0.7352706996374428,
 'TAA': 1.4993377989159233,
 'TGA': 1.445750797293688,
 'TTG': 0.37485210068210273,
 'TCG': 0.5026022954269362,
 'TAG': 0.9918828753483018,
 'TGG': 0.36987074163797057,
 'CTT': 0.5742219401008033,
 'CCT': 0.6759537482282689,
 'CAT': 0.49328446052645963,
 'CGT': 0.3517530968153242,
 'CTC': 0.7990376501273364,
 'CCC': 0.8107846312812055,
 'CAC': 0.6440567013039434,
 'CGC': 0.3227762808383639,
 'CTA': 0.3453611837060198,
 'CCA': 0.5269874729790474,
 'CAA': 0.5333872122834987,
 'CGA': 0.220499995408287,
 'CTG': 0.4779594018446893,
 'CCG': 0.5641288823751672,
 'CAG': 0.5658879417981812,
 'CGG': 0.23564145497059222,
 'ATT': 0.39025986920034234,
 'ACT': 0.6441390164369458,
 'AAT': 0.39942310656963714,
 'AGT': 0.796

In [39]:
aquifex_aeolicus_VF5.mean_RFC

{'TTT': 0.5566846941749086,
 'TCT': 0.18154772302323846,
 'TAT': 0.18376858346604297,
 'TGT': 0.3537513672835248,
 'TTC': 0.43816398579933413,
 'TCC': 0.2639698966374599,
 'TAC': 0.8097922665017615,
 'TGC': 0.3725847563481558,
 'TTA': 0.16554634895968154,
 'TCA': 0.14667316481089887,
 'TAA': 0.5077269800386349,
 'TGA': 0.3673535093367675,
 'TTG': 0.07908684579915551,
 'TCG': 0.07076865199294305,
 'TAG': 0.12491951062459755,
 'TGG': 0.836445589182228,
 'CTT': 0.2537723423020825,
 'CCT': 0.27637946655571427,
 'CAT': 0.17568350583931044,
 'CGT': 0.032323448286045484,
 'CTC': 0.2902721934006024,
 'CCC': 0.407482358028876,
 'CAC': 0.7496223537872183,
 'CGC': 0.026091998412979687,
 'CTA': 0.07304671322032018,
 'CCA': 0.14191001998050068,
 'CAA': 0.3520767933978978,
 'CGA': 0.012461352360624813,
 'CTG': 0.13698772631171888,
 'CCG': 0.16972075041237258,
 'CAG': 0.6137957114314638,
 'CGG': 0.016035822028481056,
 'ATT': 0.24507023479437787,
 'ACT': 0.24080677580965681,
 'AAT': 0.2916994456841584

## Calculate covariance matrix

Covariance matrix is defines as:

- The covariance S is a 61 X 61 covriance
matrix
    - Sij = sigma( (Xki - Xhati) * (Xkj - Xhatj)  )

k = (1,N)

i,j = (1,61)


x are gene cub

xhat are genomic mean cub

In [40]:
# init covariance matrix
covarmat = np.zeros((61,61))
# fill covariance matrix
for i,i_tag in enumerate(CODE_COVARMAT):
    for j,j_tag in enumerate(CODE_COVARMAT):
        intermediate_sum = 0
        for locust_tag in aquifex_aeolicus_VF5.genes:
            diff_A = aquifex_aeolicus_VF5[locust_tag]['RFC'][i_tag.upper()] - aquifex_aeolicus_VF5.mean_RFC[i_tag.upper()]
            diff_B = aquifex_aeolicus_VF5[locust_tag]['RFC'][j_tag.upper()] - aquifex_aeolicus_VF5.mean_RFC[j_tag.upper()]
            intermediate_sum+=diff_A * diff_B

        covarmat[i][j]= intermediate_sum

In [41]:
covarmat

array([[57.25093927,  0.73092052,  9.2300895 , ..., -1.13226894,
         0.10699351,  3.16591247],
       [ 0.73092052, 26.15905143,  2.6128969 , ..., -1.24264906,
        -3.43333813, -0.8147417 ],
       [ 9.2300895 ,  2.6128969 , 42.70442299, ..., -3.85448608,
        -4.58026888,  3.49210131],
       ...,
       [-1.13226894, -1.24264906, -3.85448608, ..., 27.65565004,
         2.04714362,  0.24058799],
       [ 0.10699351, -3.43333813, -4.58026888, ...,  2.04714362,
        24.54800398,  2.66904132],
       [ 3.16591247, -0.8147417 ,  3.49210131, ...,  0.24058799,
         2.66904132, 19.1299461 ]])

check covarmat

In [42]:
print(covarmat[1][21])
print(covarmat[21][1])
print(covarmat[50][10])
print(covarmat[10][50])

0.20001047288686286
0.20001047288686286
-0.7122612387747538
-0.7122612387747538


In [29]:
len(aquifex_aeolicus_VF5['aq_001']['sequence'])

2100

In [30]:
aquifex_aeolicus_VF5.print_gene_summary('aq_001')

Mean GC Content-> T: 46.333333333333336, 1:55.285714285714285, 2:36.0, 3:47.714285714285715
Std GC content-> T:0.7907177593645108, 1:1.0004608787268263, 2:0.8039972723685148, 3:0.005108174508592654
Relative nucleotide frequency: {'G': 0.25047619047619046, 'A': 0.3219047619047619, 'C': 0.21285714285714286, 'T': 0.21476190476190476}
Nucleotide Identity: {'A': [0.32, 0.3442857142857143, 0.30142857142857143], 'T': [0.12714285714285714, 0.2957142857142857, 0.22142857142857142], 'G': [0.38857142857142857, 0.14857142857142858, 0.21428571428571427], 'C': [0.16428571428571428, 0.21142857142857144, 0.26285714285714284]}
Dinucleotide Identity: {'AA': [0.09714285714285714, 0.12142857142857143, 0.12732474964234622], 'AG': [0.04857142857142857, 0.09428571428571429, 0.10872675250357654], 'AT': [0.12, 0.02142857142857143, 0.02861230329041488], 'AC': [0.054285714285714284, 0.10714285714285714, 0.0357653791130186], 'TA': [0.03428571428571429, 0.07571428571428572, 0.05293276108726753], 'TG': [0.015714285

## Mahalonobis distance

calculate mahalonobis distance

In [43]:
# init zeros
X = np.zeros((1,61))
Xhat = np.zeros((1,61))

for i,cds in enumerate(CODE_COVARMAT):
    X[0][i] = aquifex_aeolicus_VF5['aq_001']['RFC'][cds.upper()]
    Xhat[0][i] = aquifex_aeolicus_VF5.mean_RFC[cds.upper()]

need to check the damn mathematics

In [44]:
np.matmul(np.matmul((X-Xhat),np.linalg.inv(covarmat)),np.transpose(X-Xhat))

array([[0.11595768]])

playing around with monte carlo first:

- " generating a random sample of 10,000 sequences from the means and standard deviations of the codon usage of each genome. "

I can get the means and std deviations of codon usage but I guess I need to generate sequences from the mean and standard deviations

wtf is a monte carlo method?
- wiki: rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle.


- https://machinelearningmastery.com/monte-carlo-sampling-for-probability/:
    -   There are three main reasons to use Monte Carlo methods to randomly sample a probability distribution; they are:
        - **Estimate density**, gather samples to approximate the distribution of a target function.
        - **Approximate a quantity**, such as the mean or variance of a distribution.
        - **Optimize a function**, locate a sample that maximizes or minimizes the target function.

And we want to get a the mean and variance(standard deviation)

In [29]:
aquifex_aeolicus_VF5.mean_cub

{'TTT': 9.139729555698647,
 'TCT': 2.8068254990341277,
 'TAT': 2.4211204121056022,
 'TGT': 1.2215067611075339,
 'TTC': 7.098518995492595,
 'TCC': 4.0399227301996135,
 'TAC': 10.648422408242112,
 'TGC': 1.274307791371539,
 'TTA': 5.557630392788152,
 'TCA': 2.2549903412749517,
 'TAA': 0.5080489375402447,
 'TGA': 0.3676754668383773,
 'TTG': 2.574372182871861,
 'TCG': 1.0914359304571797,
 'TAG': 0.12491951062459755,
 'TGG': 2.9555698647778494,
 'CTT': 8.501609787508048,
 'CCT': 3.428847392144237,
 'CAT': 0.8287186091435931,
 'CGT': 0.47907276239536384,
 'CTC': 9.752092723760464,
 'CCC': 5.3927881519639405,
 'CAC': 4.047005795235029,
 'CGC': 0.3921442369607212,
 'CTA': 2.449452672247263,
 'CCA': 1.7707662588538313,
 'CAA': 2.3116548615582744,
 'CGA': 0.17707662588538314,
 'CTG': 4.585318737926594,
 'CCG': 2.21249195106246,
 'CAG': 4.141661300708306,
 'CGG': 0.24404378622021894,
 'ATT': 5.493882807469414,
 'ACT': 3.077269800386349,
 'AAT': 3.4449452672247265,
 'AGT': 2.4790727623953637,
 'AT

In [30]:
aquifex_aeolicus_VF5.std_cub

{'TTT': 7.029359764830725,
 'TCT': 2.438694251459027,
 'TAT': 2.648376928321223,
 'TGT': 1.6116530079443476,
 'TTC': 5.881745505846907,
 'TCC': 3.440930555071035,
 'TAC': 8.112856515844713,
 'TGC': 1.6552799248673526,
 'TTA': 4.772283419964373,
 'TCA': 2.1685309796482475,
 'TAA': 0.4999352104067796,
 'TGA': 0.482172394402205,
 'TTG': 2.3203875396393405,
 'TCG': 1.242366543488915,
 'TAG': 0.33062762511609406,
 'TGG': 3.2492442353387117,
 'CTT': 6.039590672364937,
 'CCT': 2.7954967508914224,
 'CAT': 1.1195272982646096,
 'CGT': 0.7802990596705658,
 'CTC': 8.102830573075925,
 'CCC': 4.981381899886754,
 'CAC': 3.712075331822215,
 'CGC': 0.6847544855872307,
 'CTA': 2.2477408039366007,
 'CCA': 1.7064690348034581,
 'CAA': 2.2683974967983143,
 'CGA': 0.4484386075343185,
 'CTG': 3.6246051602117815,
 'CCG': 2.1297085489891843,
 'CAG': 3.7546137759385494,
 'CGG': 0.5374017558113253,
 'ATT': 4.0830237560001015,
 'ACT': 2.540936904029672,
 'AAT': 3.7364516208522485,
 'AGT': 2.38631668433821,
 'ATC':

In [69]:
aquifex_aeolicus_VF5.complete_sequence

'...'

attempt to create randomiser with weights.. closest thing available is Fabox

- fabox: https://birc.au.dk/~palle/php/fabox/random_sequence_generator.php
    - problem is that fabox uses fraction of GACT and not codon usage bias   

taken from and modified:
- https://stackoverflow.com/questions/21205836/generating-random-sequences-of-dna

In [80]:
def weightedchoice(items): # this doesn't require the numbers to add up to 100
    return choice("".join(x * y for x, y in items))

In [81]:

def random_dna_sequence(length):
  if length%3!=0:
    raise ValueError('length needs to be disvisible by 3!')
  list_of_cds = choices([str(y) for y,x in aquifex_aeolicus_VF5.cub.items()], weights=[int(x) for y,x in aquifex_aeolicus_VF5.cub.items()], k=length)
  DNA=""
  for cds in list_of_cds:
    DNA+=cds
  return DNA

In [66]:
import numpy as np
from random import choices,choice

In [67]:
def random_dna_sequence_v2(length):
  if length%3!=0:
    raise ValueError('length needs to be disvisible by 3!')
  
  weights = []
  for i in CODE_COVARMAT:    
      mu, sigma = aquifex_aeolicus_VF5.mean_cub[i.upper()], aquifex_aeolicus_VF5.std_cub[i.upper()]
      s = np.random.normal(mu, sigma, 1)
      weights.append(s)
  list_of_cds = choices([cds.upper() for cds in CODE_COVARMAT.keys()], weights=weights, k=length)
  DNA=""
  for cds in list_of_cds:
    DNA+=cds
  return DNA

In [68]:
# print 10 thousand sequences
list_of_random_dnas = []
for i in range(10000):
    list_of_random_dnas.append(random_dna_sequence_v2(300))

In [69]:
len(list_of_random_dnas)

10000

In [70]:
list_of_random_dnas[0]

'CTAGGACTTGGTGTTGAGGTTTTCAGGAGATTTATATTTGATGCTACAGTACTGTTAAATTCTTGGTGGGTAAGAGGTATGTTCGTGGATGTTGTTGTTGATATTTCTTCATTCTTTCTTTATCTAGGAAATGGTAGGTCAGGAGTACTTGTTGCTCAGCTTGGTTTCCTTTTTCCAAATATTTTCATGGATGGAGACCTTGGAGACGATATACTTATTAATTTTTACCTTGTGTTTACTGATGTTGTTGGGTCAATTCTAATTGCTTTGTATTATTGGTTAGTTCTATGGTATGGTGAGGATTTTGGTATGTCTGTTATTTTTGTATTCTTCGCTTTCTTTAAGCCAGTACTTGTAACGAATCTTCTTGACGGTTCTTACCTTTTTTTTAAGGTTGGGGCCTTCCAGGTTCTAACAGACGATAGAAGAGCCGCCCTGACTAGGTTTACATCAGATAGAGGTGGTTACGTAAAGTCCGTAGTTAAGAGAACAGACTTATTTGACGACGCCTCTTCTAAGGGAGATTACTTTCTTGACATATACTTTGTAGACGACTGGACTGGAAATATTGGTAATTACATTCTTTCATCCGCCGTAGACGCTGTATGGATTGTATGGTACAGGCTGGACAATGATCTTTTCGACGCTGTTGGTTGGAATGATGTTGGCGGTTACAAGTTTTATTCAAATGTTACAGATTATTTAATTATTGTTATTACAAATCCAGATGACGCCATGTTCGTATTTCTGGTTTGGGGATACCTTATGGTACAGGACGCTGTTGGCGGTTTCACAGGACCACTTGGGGGAAAGTTTGACTCCGCGGTGAAGTTATGCTACATAGTGGGAGACTTCAAGAGATGGAAGCTTGGGTCTGGTGACCTGTTCGATCTTTCATTC'

In [71]:
list_of_random_dnas[7]

'GAAGGCTCAGAAGACAGGAAAGCCCAGGAGCATATGAGCAACAGGTATAAGGACGATATAGAAAGTGACTTTAAGCTCGAGGAAAGAATAGGCGTGGTTTATGAATTAGATGTTACTGCTGCTGTTAGGGATAACGACGAAGAATCAGTTTATGAGTTTGATTATTTACAGTTTGAGGACTATGGAGGGGTGGGGAAATTTTCGCGTAAGATAAGGTTCGATAACAAAAACGAGGAGCTTCCGGAAACGGCTATAGCTAGCGACGAGTACGAAAGGTTTTTTACTGAACTTGACTACGTCCAGAACTTTTTTGCAGGTGAAGCTGAGGATGAGGGATTTACAAGACTCGGGTTCGTTACAGCCGTATTCTATGCATTCATTTTTATACATGCCTTCTTTAAAAACTTAGAAGGTATAGAGGCAGAGGAGGAGTGCGCGATAGAGGACTACGCGACTGACGACCGTTATGTTATGAAGTTTAACGTCGTTGGGGAGTTCTTTAGATACGCCAGCGAACTCGAGTTTGGCCGTCTCATGAATGCGAGGCCGTTTCTCTTGCTTAACGGGCAGCCTGAAGAGTTTGGAGACCATAACGATGAGTTCTATTTTAAAGCAAGCGTTTTCACTGAGGTTATATACAAAGAAACTGCAGGGTCTAGATACTTCGAGTTCTTTGTTGACGAATGCGAATTCGTTGACGTAATAAGTGGTGAATTCGTTGAACGTAACGTTGGATTCGTCTCAGAGAGAAAATTCTCTTTTTTCGAAGCTTACGAAGGAACTGATGAGGTTAGATCGGTTAACGTCGGGAACGTGGAAGGGTTCTTCACTGAGGCAAACGACGACGCTTATGCAATGAGATGCTTCAAAGAGGATACGAGGATATTCGACTTTATGAAT'

In [72]:
from data.utils.NCBI.metrics.calc_cub import calc_cub

In [73]:
# calculate cub for each generated sequences

list_of_cubs = []
for i in range(10000):
    list_of_cubs.append(calc_cub(list_of_random_dnas[i]))

In [74]:
# calculate cub means of 10 000 random sequences
mean_cub_gen = {}
for cds in CODE_COVARMAT:
    mean_cub_gen[cds.upper()] = 0

for i in range(10000):
    curr_cub = list_of_cubs[i]
    for cds in CODE_COVARMAT:
        mean_cub_gen[cds.upper()] += curr_cub[cds.upper()]

In [75]:
for cds in mean_cub_gen:
    mean_cub_gen[cds] = mean_cub_gen[cds]/(10000)

In [76]:
mean_cub_gen

{'TTT': 8.8446,
 'TCT': 2.5268,
 'TAT': 2.3358,
 'TGT': 1.2006,
 'TTC': 6.8595,
 'TCC': 3.6941,
 'TAC': 10.1919,
 'TGC': 1.2778,
 'TTA': 5.4462,
 'TCA': 2.1137,
 'TTG': 2.4087,
 'TCG': 1.0486,
 'TGG': 2.9842,
 'CTT': 7.9307,
 'CCT': 3.2637,
 'CAT': 0.8449,
 'CGT': 0.5126,
 'CTC': 9.3775,
 'CCC': 5.003,
 'CAC': 3.7755,
 'CGC': 0.4299,
 'CTA': 2.2899,
 'CCA': 1.5983,
 'CAA': 2.2514,
 'CGA': 0.2438,
 'CTG': 4.2888,
 'CCG': 2.0379,
 'CAG': 3.9885,
 'CGG': 0.3183,
 'ATT': 5.138,
 'ACT': 3.0001,
 'AAT': 3.4649,
 'AGT': 2.2844,
 'ATC': 2.8895,
 'ACC': 3.5086,
 'AAC': 7.5198,
 'AGC': 2.3105,
 'ATA': 13.9523,
 'ACA': 2.6975,
 'AAA': 13.4957,
 'AGA': 5.6885,
 'ATG': 5.5426,
 'ACG': 3.7009,
 'AAG': 14.4797,
 'AGG': 7.7626,
 'GTT': 9.166,
 'GCT': 4.7936,
 'GAT': 4.9158,
 'GGT': 4.6519,
 'GTC': 2.5567,
 'GCC': 3.6568,
 'GAC': 8.2528,
 'GGC': 2.3917,
 'GTA': 7.5124,
 'GCA': 4.9794,
 'GAA': 18.7295,
 'GGA': 10.0862,
 'GTG': 4.5486,
 'GCG': 4.1267,
 'GAG': 10.0205,
 'GGG': 3.0886}